In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pytorch_dataset
import pytorch_datamodule

import matplotlib.pyplot as plt
import pandas as pd

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [58]:
df = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz3d_and_gz_desi_matches.csv')[:100]

# temp, until I update master df
df['segmap_json_loc'] = df['local_gz3d_fits_loc'].str.replace('/fits_gz/', '/segmaps/', regex=False).str.replace('.fits.gz', '.json', regex=False)
df['local_desi_jpg_loc'] = df.apply(lambda x: f'data/desi/jpg/{x["brickid"]}/{x["brickid"]}_{x["objid"]}.jpg', axis=1)


In [59]:
df.columns.values

array(['ra_manga', 'dec_manga', 'mangaid', 'iauname', 'ifudesignsize',
       'manga_tileid', 'nsa_id', 'explorer_link',
       'gz_total_classifications', 'gz_bar_votes', 'gz_spiral_votes',
       'sample', 'local_gz3d_fits_loc', 'best_match', 'sky_separation',
       'ra_desi', 'dec_desi', 'brickid', 'objid', 'dr8_id', 'mag_r',
       'mag_g', 'mag_z', 'est_dr5_pixscale', 'segmap_json_loc',
       'local_desi_jpg_loc'], dtype=object)

In [60]:
gz_desi = pd.read_parquet('/Users/user/Downloads/gz_desi_deep_learning_catalog_friendly.parquet',
                          columns=[
                              'dr8_id',
                              'smooth-or-featured_featured-or-disk_fraction',
                              'has-spiral-arms_yes_fraction',
                              'spiral-arm-count_2_fraction'
                            ])

In [61]:
gz_desi = gz_desi[gz_desi['smooth-or-featured_featured-or-disk_fraction'] > 0.5]
gz_desi = gz_desi[gz_desi['has-spiral-arms_yes_fraction'] > 0.5]
gz_desi = gz_desi[gz_desi['spiral-arm-count_2_fraction'] > 0.3]

In [62]:

df = pd.merge(df, gz_desi, on='dr8_id', validate='one_to_one', how='inner')

In [63]:
len(df)

7

In [64]:
transform = A.Compose(
    [
        A.CenterCrop(250, 250)
    ],
    additional_targets={'spiral_mask': 'image', 'bar_mask': 'image'}
)
# transform = None

In [65]:
# n_examples = 7

# dataset = pytorch_dataset.SegmentationGalaxyDataset(df.sample(n_examples), transform=transform)

# fig, rows = plt.subplots(ncols=3, nrows=n_examples, figsize=(7, 2*n_examples))


# for galaxy_n in range(n_examples):

#     (ax0, ax1, ax2) = rows[galaxy_n]

#     image, segmap_dict = dataset[galaxy_n]
   
#     ax0.imshow(segmap_dict['spiral_mask'])
#     ax1.imshow(segmap_dict['bar_mask'])
#     ax2.imshow(image)  # default target keyed as image
#     ax0.axis('off')
#     ax1.axis('off')
#     ax2.axis('off')

# plt.show()

## Check DataModule 

In [66]:
transform = A.Compose(
    [
        A.CenterCrop(250, 250),
        ToTensorV2()
    ],
    additional_targets={'spiral_mask': 'image', 'bar_mask': 'image'}
)

In [67]:
datamodule = pytorch_datamodule.SegmentationDataModule(
    train_catalog=df[:2],
    val_catalog=df[2:4],
    test_catalog=df[4:6],
    batch_size=2,
    num_workers=1,
    transform=transform
)
datamodule.setup('fit')

In [68]:
for batch in datamodule.train_dataloader():
    print(len(batch))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/user/mambaforge/envs/dev/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/user/mambaforge/envs/dev/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/Users/user/mambaforge/envs/dev/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 160, in default_collate
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/Users/user/mambaforge/envs/dev/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 160, in <dictcomp>
    return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
  File "/Users/user/mambaforge/envs/dev/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 141, in default_collate
    r